In [1]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))

ableIngre = []
RecipeVector = []
Recipe_cate_ingre_vector = []

from scipy import spatial
# input two vector arrays
import re

def dfmaking():
    file = "Id2Recipe.csv"
    file_match = "Recipecategories.csv"
    Id = []
    Recipe = []
    with open(file,'r') as f:
        lines = f.readlines()
        for line in lines:
            Id.append(line.split('\t')[0])
            Recipe.append(line.split('\t')[1].strip('\n'))
    Category1 = []
    Category2 = []
    with open(file_match,'r') as f:
        lines = f.readlines()
        for line in lines:
            Category1.append(line.split('\t')[1])
            Category2.append(line.split('\t')[2].strip('\n'))
    df = pd.DataFrame({"id":Id,"recipe":Recipe,"from":Category1, "what":Category2})
    df['result'] = df['what'].apply(lambda x : [item for item in x.split(',')]).apply(lambda y: " ".join(y).strip("[").strip("]").strip("\""))
    df['result'] = df['result'].apply(lambda x : x.strip("\“").strip("\”").strip("\"").strip("\”").strip("\"").strip("\” “"))
    df['result'] = df['result'].apply(lambda x : x.replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“",""))
    df['result'] = df['result'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(",","").replace("” “"," "))
    size = len(df)
    df['from'] = df['from'].apply(lambda x : x.strip('[').strip(']').replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“","").replace(" ,","").replace(",",""))
    df['from'] = df['from'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(","," ").replace("” “"," ").replace("\""," "))
    for i in range(size):
        df['result'][i] = str(df['result'][i]) + str(df['from'][i])
        
    return df


def calculate_similarity_vectors(vector1, vector2): 
    return 1 - spatial.distance.cosine(vector1.tolist(), vector2.tolist())


def generateUserPreferenceVector(df,preferenceJSON):
    likeList = preferenceJSON['like']
    likeList = list(filter(lambda x : x!= None,likeList))
    vector = np.zeros(661)
    df1 =  df[df["recipe"].isin(likeList)]
    size = len(df1)
    for i in range(size):
        vector += df1.iloc[i]['vector']
    return vector


def import_vector_ingre():
    with open("vector_ingre1.csv","r") as f:
        ingre_vector = []
        ingre_vectori = []
        a = f.readlines()
    size = len(a)
    for i in range(size-1):
        ingre_vectori.append(a[i])
        if a[i+1][0] == '_':
            ingre_vector.append(ingre_vectori)
            ingre_vectori = []
        elif i+1 == size-1:
            ingre_vectori.append(a[i+1])

    ingre_vector.append(ingre_vectori)
    ingre_Vector = {}
    for item in ingre_vector:
        r = ""
        for row in item:
            if row[0] == "_":
                r +=  item[0].split(":")[1].strip('[')
            else:
                r += row
        r = list(map(lambda x : float(x),r.strip().replace("]","").split()))
        ingre_Vector[str("".join(re.compile("[가-힣]").findall(item[0])))] = r
    return ingre_Vector



def calculate_vector_Recipe(df):
    ingre_Vector = import_vector_ingre()
    
    f = open('my_new_csv551.csv', 'r', encoding='utf-8')
    recipeList = csv.reader(f)
    l = []
    for i in recipeList:
        li = i[5].strip('[').strip(']')
        l.append(li.split(","))
    res = []
    for i in l:
        small = []
        for j in i:
            small.append(j.replace('"', ''))
        res.append(small)
    #ableIngre = []
    #RecipeVector = []
    
    
    
    category_vector = count_vector.fit_transform(df['result'])
    for ingrelist in res:
        RecipeVector_i = np.zeros(50)
        for item in ingrelist:
            if item in ingre_Vector.keys():
                ableIngre.append(item)
                RecipeVector_i += np.array(ingre_Vector[item])
        RecipeVector.append(RecipeVector_i)
    #Recipe_cate_ingre_vector = []
    length = len(df)
    for i in range(length):
        Recipe_cate_ingre_vector.append(category_vector[i].toarray().tolist()[0]+RecipeVector[i].tolist())
    df['vector'] = pd.DataFrame([np.array(Recipe_cate_ingre_vector[i])] for i in range(length))
    return df


def get_recommend_by_userVector(df, JSON = {"like":["짜장면","짬뽕"]}, top=20):
    vector = generateUserPreferenceVector(df,JSON)
    df['similarity'] = df['vector'].apply(lambda x : calculate_similarity_vectors(vector, x))
    df = df.sort_values(by='similarity',ascending=False)
    #return [i for i in df[:top].id]
    return df[:top]


'''
def get_recommend_by_userVector(df, vector, top=20):
    df['similarity'] = df['vector'].apply(lambda x : calculate_similarity_vectors(vector, x))
    df = df.sort_values(by='similarity',ascending=False)
    #return [i for i in df[:top].id]
    return df[:top]
'''

df = calculate_vector_Recipe(dfmaking())
get_recommend_by_userVector(df, top=20)
#get_recommend_by_userVector(df, np.array(Recipe_cate_ingre_vector[215]) , top=20)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


,id,recipe,from,what,result,vector,similarity
498,5398,짜장면,중식,"[""가공식품 요리”,""면/만두요리"",""밀가루""]",가공식품면/만두 밀가루 중식,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.918190
490,5390,짬뽕,중식,"[""면/만두요리"",”해산물"",""양념""]","면/만두""해산물 양념 중식","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.908465
229,5696,팽이오징어볶음,,"[""채소류 요리""]",채소류,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.788772
215,5712,육즙가득 짬뽕 군만두,한식,"[""가공식품 요리""]",가공식품 한식,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.788020
189,5740,여수 해물삼합,,"[""채소류 요리”, “해산물”, “육류 요리”]",채소류 해산물 육류,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.779788
284,5624,매운순대볶음,,"[“양념”, “채소류 요리”]",양념 채소류,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.765898
172,5759,중국당면볶이,퓨전,"[“가공식품 요리”, “면/만두요리”]",가공식품 면/만두 퓨전,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.757963
195,5733,대구 중화식비빔밥,한식 중식,"[""채소류 요리"",""가공식품 요리”, “쌀”]",채소류 가공식품 쌀 한식 중식,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.744779
270,5642,장어 잡채,한식,"[""채소류 요리”, “해산물”, “양념”, “버섯류”]",채소류 해산물 양념 버섯류 한식,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.738049
80,5876,매운 제육볶음,한식,[“육류 요리”],육류 한식,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.737738


In [46]:
aa = ["에그샌드위치",
"짬뽕",
"회",
"미역국2",
"닭가슴살 샐러드",
"비프스테이크",
"김치찌개2",
"파스타",
"통삼겹구이",
"짜장면",
"두부조림",
"계란말이2",
"돈까스덮밥"]
df = dfmaking()
vector = np.zeros(645)
df1 =  df[df["recipe"].isin(aa)]
s = len(df1)
for i in range(s):
    print(df1.iloc[i]['recipe'])

에그샌드위치
짬뽕
회
미역국2
닭가슴살 샐러드
비프스테이크
김치찌개2
파스타
통삼겹구이
짜장면
두부조림
계란말이2
돈까스덮밥


In [99]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))

ableIngre = []
RecipeVector = []
Recipe_cate_ingre_vector = []

from scipy import spatial
# input two vector arrays
import re

def dfmaking():
    file = "Id2Recipe.csv"
    file_match = "Recipecategories.csv"
    Id = []
    Recipe = []
    with open(file,'r') as f:
        lines = f.readlines()
        for line in lines:
            Id.append(line.split('\t')[0])
            Recipe.append(line.split('\t')[1].strip('\n'))
    Category1 = []
    Category2 = []
    with open(file_match,'r') as f:
        lines = f.readlines()
        for line in lines:
            Category1.append(line.split('\t')[1])
            Category2.append(line.split('\t')[2].strip('\n'))
    df = pd.DataFrame({"id":Id,"recipe":Recipe,"from":Category1, "what":Category2})
    df['result'] = df['what'].apply(lambda x : [item for item in x.split(',')]).apply(lambda y: " ".join(y).strip("[").strip("]").strip("\""))
    df['result'] = df['result'].apply(lambda x : x.strip("\“").strip("\”").strip("\"").strip("\”").strip("\"").strip("\” “"))
    df['result'] = df['result'].apply(lambda x : x.replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“",""))
    df['result'] = df['result'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(",","").replace("” “"," "))
    size = len(df)
    df['from'] = df['from'].apply(lambda x : x.strip('[').strip(']').replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“","").replace(" ,","").replace(",",""))
    df['from'] = df['from'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(","," ").replace("” “"," ").replace("\""," "))
    for i in range(size):
        df['result'][i] = str(df['result'][i]) + str(df['from'][i])
        
    return df


def calculate_similarity_vectors(vector1, vector2): 
    return 1 - spatial.distance.cosine(vector1.tolist(), vector2.tolist())


def generateUserPreferenceVector(df,preferenceJSON):
    likeList = preferenceJSON['like']
    likeList = list(filter(lambda x : x!= None,likeList))
    vector = np.zeros(661)
    df1 =  df[df["recipe"].isin(likeList)]
    size = len(df1)
    for i in range(size):
        vector += df1.iloc[i]['vector']
    return vector



def import_vector_ingre():
    with open("vector_ingre1.csv","r") as f:
        ingre_vector = []
        ingre_vectori = []
        a = f.readlines()
    size = len(a)
    for i in range(size-1):
        ingre_vectori.append(a[i])
        if a[i+1][0] == '_':
            ingre_vector.append(ingre_vectori)
            ingre_vectori = []
        elif i+1 == size-1:
            ingre_vectori.append(a[i+1])

    ingre_vector.append(ingre_vectori)
    ingre_Vector = {}
    for item in ingre_vector:
        r = ""
        for row in item:
            if row[0] == "_":
                r +=  item[0].split(":")[1].strip('[')
            else:
                r += row
        r = list(map(lambda x : float(x),r.strip().replace("]","").split()))
        ingre_Vector[str("".join(re.compile("[가-힣]").findall(item[0])))] = r
    return ingre_Vector



def calculate_vector_Recipe(df):
    ingre_Vector = import_vector_ingre()
    
    f = open('my_new_csv551.csv', 'r', encoding='utf-8')
    recipeList = csv.reader(f)
    l = []
    for i in recipeList:
        li = i[5].strip('[').strip(']')
        l.append(li.split(","))
    res = []
    for i in l:
        small = []
        for j in i:
            small.append(j.replace('"', ''))
        res.append(small)
    #ableIngre = []
    #RecipeVector = []
    
    
    
    category_vector = count_vector.fit_transform(df['result'])
    for ingrelist in res:
        RecipeVector_i = np.zeros(50)
        for item in ingrelist:
            if item in ingre_Vector.keys():
                ableIngre.append(item)
                RecipeVector_i += np.array(ingre_Vector[item])
        RecipeVector.append(RecipeVector_i)
    #Recipe_cate_ingre_vector = []
    length = len(df)
    for i in range(length):
        Recipe_cate_ingre_vector.append(category_vector[i].toarray().tolist()[0]+RecipeVector[i].tolist())
    df['vector'] = pd.DataFrame([np.array(Recipe_cate_ingre_vector[i])] for i in range(length))
    return df

def get_recommend_by_userVector(df, possible_recipe_id_list, top=20,JSON = {"like":["짜장면","짬뽕"]}):
    df1 = df.iloc[possible_recipe_id_list]
    vector = generateUserPreferenceVector(df,JSON)
    df1['similarity'] = df1['vector'].apply(lambda x : calculate_similarity_vectors(vector, x))
    df1 = df1.sort_values(by='similarity',ascending=False)
    #return [i for i in df[:top].id]
    return df1[:top]


df = calculate_vector_Recipe(dfmaking()) # preprocessing
#print(get_recommend_by_userVector(df, np.array(Recipe_cate_ingre_vector[373]) , [0,1,2,3,4,5,100],top=20).id) #input = instead of np.array(...), userVector 
print(get_recommend_by_userVector(df,  [1,20,2,3,4,5,100],top=10))

       id                 recipe        from  \
1    5979              베이컨 숙주 볶음         퓨전    
3    5977                소고기 장조림          한식   
20   5960                    어묵탕          한식   
5    5975            로스트 치킨 스테이크      한식 퓨전    
2    5978         베트남 국민소스 '느억맘'      동남아/인도   
4    5976  연어 스테이크 & 레몬 머스터드 드레싱   양식 이국적인맛    
100  5843                  토마토빙수   일식 양식 퓨전    

                                  what                        result  \
1           [“육류 요리”, “채소류 요리”, “버섯류”]                육류 채소류 버섯류 퓨전    
3                      [“육류 요리”, “양념”]                      육류 양념 한식   
20                    [“가공식품”, “튀김요리”]                    가공식품 튀김 한식   
5                            [“육류 요리”]                    육류  한식 퓨전    
2                           [“채소류 요리”]                   채소류  동남아/인도   
4                            [“육류 요리”]                 육류  양식 이국적인맛    
100  ["채소류 요리","달걀/유제품 요리”, “음료/차/커피”]  채소류 달걀/유제품 음료/차/커피 일식 양식 퓨전    

                                      

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [80]:
df2 = calculate_vector_Recipe(dfmaking())

In [81]:
df3 = df2.iloc[[0,1,2,3,4,5,100]]

In [82]:
df3

,id,recipe,from,what,result,vector
0,5980,단호박 스프,양식,[“채소류 요리”],채소류 양식,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,5979,베이컨 숙주 볶음,퓨전,"[“육류 요리”, “채소류 요리”, “버섯류”]",육류 채소류 버섯류 퓨전,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5978,베트남 국민소스 '느억맘',동남아/인도,[“채소류 요리”],채소류 동남아/인도,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,5977,소고기 장조림,한식,"[“육류 요리”, “양념”]",육류 양념 한식,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,5976,연어 스테이크 & 레몬 머스터드 드레싱,양식 이국적인맛,[“육류 요리”],육류 양식 이국적인맛,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,5975,로스트 치킨 스테이크,한식 퓨전,[“육류 요리”],육류 한식 퓨전,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
100,5843,토마토빙수,일식 양식 퓨전,"[""채소류 요리"",""달걀/유제품 요리”, “음료/차/커피”]",채소류 달걀/유제품 음료/차/커피 일식 양식 퓨전,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [85]:
vector = generateUserPreferenceVector(df3,{"like":["짜장면","짬뽕"]})

In [88]:
vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [90]:
v = np.zeros(661)
for i in range(len(df3)):
    v += df3.iloc[i]['vector']
v

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [2]:
df = dfmaking()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))
df = dfmaking()
category_vector = count_vector.fit_transform(df['result'])
category_vector.shape

(502, 611)

In [12]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
import numpy as np
import csv
import json
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))

ableIngre = []
RecipeVector = []
Recipe_cate_ingre_vector = []

from scipy import spatial
# input two vector arrays
import re

def dfmaking():
    file = "Id2Recipe.csv"
    file_match = "Recipecategories.csv"
    Id = []
    Recipe = []
    with open(file,'r') as f:
        lines = f.readlines()
        for line in lines:
            Id.append(line.split('\t')[0])
            Recipe.append(line.split('\t')[1].strip('\n'))
    Category1 = []
    Category2 = []
    with open(file_match,'r') as f:
        lines = f.readlines()
        for line in lines:
            Category1.append(line.split('\t')[1])
            Category2.append(line.split('\t')[2].strip('\n'))
    df = pd.DataFrame({"id":Id,"recipe":Recipe,"from":Category1, "what":Category2})
    df['result'] = df['what'].apply(lambda x : [item for item in x.split(',')]).apply(lambda y: " ".join(y).strip("[").strip("]").strip("\""))
    df['result'] = df['result'].apply(lambda x : x.strip("\“").strip("\”").strip("\"").strip("\”").strip("\"").strip("\” “"))
    df['result'] = df['result'].apply(lambda x : x.replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“",""))
    df['result'] = df['result'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(",","").replace("” “"," "))
    size = len(df)
    df['from'] = df['from'].apply(lambda x : x.strip('[').strip(']').replace("요리","").replace(" ”","").replace(" “","").replace("”","").replace("“","").replace(" ,","").replace(",",""))
    df['from'] = df['from'].apply(lambda x : x.replace(" \" \""," ").replace("\" \""," ").replace(" \"","").replace(","," ").replace("” “"," ").replace("\""," "))
    for i in range(size):
        df['result'][i] = str(df['result'][i]) + str(df['from'][i])
        
    return df


def calculate_similarity_vectors(vector1, vector2): 
    return 1 - spatial.distance.cosine(vector1.tolist(), vector2.tolist())


def generateUserPreferenceVector(df,preferenceJSON):
    likeList = preferenceJSON['like']
    likeList = list(filter(lambda x : x!= None,likeList))
    vector = np.zeros(661)
    df1 =  df[df["recipe"].isin(likeList)]
    size = len(df1)
    for i in range(size):
        vector += df1.iloc[i]['vector']
    return vector



def import_vector_ingre():
    with open("vector_ingre1.csv","r") as f:
        ingre_vector = []
        ingre_vectori = []
        a = f.readlines()
    size = len(a)
    for i in range(size-1):
        ingre_vectori.append(a[i])
        if a[i+1][0] == '_':
            ingre_vector.append(ingre_vectori)
            ingre_vectori = []
        elif i+1 == size-1:
            ingre_vectori.append(a[i+1])

    ingre_vector.append(ingre_vectori)
    ingre_Vector = {}
    for item in ingre_vector:
        r = ""
        for row in item:
            if row[0] == "_":
                r +=  item[0].split(":")[1].strip('[')
            else:
                r += row
        r = list(map(lambda x : float(x),r.strip().replace("]","").split()))
        ingre_Vector[str("".join(re.compile("[가-힣]").findall(item[0])))] = r
    return ingre_Vector



def calculate_vector_Recipe(df):
    ingre_Vector = import_vector_ingre()
    
    f = open('my_new_csv551.csv', 'r', encoding='utf-8')
    recipeList = csv.reader(f)
    l = []
    for i in recipeList:
        li = i[5].strip('[').strip(']')
        l.append(li.split(","))
    res = []
    for i in l:
        small = []
        for j in i:
            small.append(j.replace('"', ''))
        res.append(small)
    #ableIngre = []
    #RecipeVector = []
    
    
    
    category_vector = count_vector.fit_transform(df['result'])
    for ingrelist in res:
        RecipeVector_i = np.zeros(50)
        for item in ingrelist:
            if item in ingre_Vector.keys():
                ableIngre.append(item)
                RecipeVector_i += np.array(ingre_Vector[item])
        RecipeVector.append(RecipeVector_i)
    #Recipe_cate_ingre_vector = []
    length = len(df)
    for i in range(length):
        Recipe_cate_ingre_vector.append(category_vector[i].toarray().tolist()[0]+RecipeVector[i].tolist())
    df['vector'] = pd.DataFrame([np.array(Recipe_cate_ingre_vector[i])] for i in range(length))
    return df

def get_recommend_by_userVector(df, possible_recipe_id_list, top=20,JSON = {"like":["짜장면","짬뽕"]}):
    df1 = df[df['id'].isin(list(map(lambda x: str(x),possible_recipe_id_list)))]
    vector = generateUserPreferenceVector(df,JSON)
    df1['similarity'] = df1['vector'].apply(lambda x : calculate_similarity_vectors(vector, x))
    df1 = df1.sort_values(by='similarity',ascending=False)
    #return [i for i in df[:top].id]
    #return df1[:top]
    size = len(df1)
    retId =  []
    for i in range(size):
        retId.append(df1.iloc[i].id)
    return json.dumps({"id":retId})
df = calculate_vector_Recipe(dfmaking()) # preprocessing
#print(get_recommend_by_userVector(df, np.array(Recipe_cate_ingre_vector[373]) , [0,1,2,3,4,5,100],top=20).id) #input = instead of np.array(...), userVector 
print(get_recommend_by_userVector(df,  [5394,5657,5797,5842],top=20))




{"id": ["5842", "5394", "5797", "5657"]}


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
